In [ ]:
%%capture
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4 
!pip install --no-deps trl==0.22.2
!pip install sacrebleu

In [ ]:
import os
import torch
import sacrebleu
import numpy as np
import unicodedata
import re
import html
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

# --- CẤU HÌNH ĐƯỜNG DẪN & THAM SỐ CUỐI CÙNG ---
max_seq_length = 512 
dtype = None
load_in_4bit = True

VALID_CSV_PATH = "/kaggle/input/train-final/valid_clean.csv"

# TEST_EN_PATH = "/kaggle/input/d/hehewelldone/vlsp-medical/MedicalDataset_VLSP/public_test.en.txt"
# TEST_VI_PATH = "/kaggle/input/d/hehewelldone/vlsp-medical/MedicalDataset_VLSP/public_test.vi.txt"
CHECKPOINT_PATH = "/kaggle/input/medical-training/outputs_adapter_en_vi/checkpoint-1000" # Checkpoint cuối cùng

MAX_SAFE_NEW_TOKENS = 331
BATCH_SIZE = 32 # Tối ưu hóa tốc độ Inference 

In [ ]:
import pandas as pd

df_valid = pd.read_csv(VALID_CSV_PATH)

assert {"en", "vi"}.issubset(df_valid.columns)

valid_en = df_valid["en"].astype(str).tolist()
valid_vi = df_valid["vi"].astype(str).tolist()

# EN → VI
vi_references = [[v.strip()] for v in valid_vi]

# VI → EN
en_references = [[e.strip()] for e in valid_en]


print("Valid size:", len(valid_en))


In [ ]:
#--- 1. LOAD MODEL VÀ ADAPTERS ---
print(f" Đang tải model từ: {CHECKPOINT_PATH}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = CHECKPOINT_PATH, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "cuda:0",
)
FastLanguageModel.for_inference(model)
print(" Model đã sẵn sàng cho Inference!")

In [ ]:
# # --- 2. LOAD VÀ LÀM SẠCH DATA ---
# def preprocess_text(text):
#     if not isinstance(text, str): return ""
#     text = html.unescape(text)
#     text = re.sub(r'<[^>]+>', '', text)
#     text = re.sub(r'(?:https?://|www\.)\S+', '', text)
#     text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
#     text = unicodedata.normalize('NFC', text)
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# print(" Đang load và làm sạch Public Test...")
# with open(TEST_EN_PATH, 'r', encoding='utf-8') as f: en_data = [preprocess_text(l) for l in f]
# with open(TEST_VI_PATH, 'r', encoding='utf-8') as f: vi_data = [preprocess_text(l) for l in f]

# # Reference phải là list of list cho sacrebleu
# en_references = [[l.strip()] for l in en_data]
# vi_references = [[l.strip()] for l in vi_data]
# print(f"Đã load {len(en_data)} câu Test (sẵn sàng tính toán).")

In [ ]:
#  PHASE 3: TÍNH EVAL LOSS CUỐI CÙNG (1.3233)
# ==============================================================================

sys_prompt_en_vi = (
    "Bạn là một biên dịch viên y tế chuyên nghiệp. "
    "Nhiệm vụ của bạn là dịch chính xác văn bản y khoa từ tiếng Anh sang tiếng Việt, "
    "đảm bảo văn phong khoa học và thuật ngữ chính xác."
)
# sys_prompt_vi_en = (
#     "You are a professional medical translator. "
#     "Your task is to accurately translate the following Vietnamese medical text into English. "
#     "Ensure correct medical terminology and academic style."
# )

def calculate_eval_loss(en_data, vi_data):
    formatted_texts = []
    for en, vi in zip(en_data, vi_data):
        # Chiều 1: En -> Vi
        msg_en_vi = [{"role": "system", "content": sys_prompt_en_vi}, {"role": "user", "content": en}, {"role": "assistant", "content": vi}]
        formatted_texts.append(tokenizer.apply_chat_template(msg_en_vi, tokenize=False, add_generation_prompt=False))
        
        # # Chiều 2: Vi -> En
        # msg_vi_en = [{"role": "system", "content": sys_prompt_vi_en}, {"role": "user", "content": vi}, {"role": "assistant", "content": en}]
        # formatted_texts.append(tokenizer.apply_chat_template(msg_vi_en, tokenize=False, add_generation_prompt=False))

    print(" Đang Tokenize toàn bộ tập valid để tính Loss...")
    inputs = tokenizer(
        formatted_texts,
        padding = True,
        truncation = True,
        max_length = max_seq_length,
        return_tensors = "pt",
    )

    total_loss = 0
    num_batches = 0
    model.eval()

    with torch.no_grad():
        # Lặp qua từng batch
        for i in range(0, len(inputs.input_ids), BATCH_SIZE):
            # 1. Cắt batch từ CPU
            batch_input_ids = inputs.input_ids[i : i + BATCH_SIZE]
            batch_attention_mask = inputs.attention_mask[i : i + BATCH_SIZE]
            
            # 2. Bây giờ mới đẩy batch nhỏ này lên GPU
            batch_input_ids = batch_input_ids.to("cuda")
            batch_attention_mask = batch_attention_mask.to("cuda")
            
            # 3. Tạo labels (-100 cho padding)
            batch_labels = batch_input_ids.clone()
            batch_labels[batch_input_ids == tokenizer.pad_token_id] = -100
            
            # 4. Tính toán
            outputs = model(
                input_ids=batch_input_ids,
                attention_mask=batch_attention_mask,
                labels=batch_labels
            )
            
            total_loss += outputs.loss.item()
            num_batches += 1
            
            # Dọn dẹp ngay
            del batch_input_ids, batch_attention_mask, batch_labels, outputs
            
            if num_batches % 20 == 0:
                 print(f"   -> Batch {num_batches}: Loss hiện tại ~ {total_loss/num_batches:.4f}", end='\r')

    if num_batches == 0: return 0, 0
    
    final_eval_loss = total_loss / num_batches
    return final_eval_loss, torch.exp(torch.tensor(final_eval_loss)).item()

# Chạy thôi
eval_loss, ppl = calculate_eval_loss(valid_en, valid_vi)
print("\n" + "="*40)
print(f" KẾT QUẢ EVAL LOSS (En->Vi Specialist)")
print(f" Eval Loss: {eval_loss:.4f}")
print(f" Perplexity: {ppl:.4f}")
print("="*40)

In [ ]:
#  PHASE 4: TÍNH BLEU SCORE CUỐI CÙNG (SỬ DỤNG TỐI ƯU TỐC ĐỘ)
# ==============================================================================
tokenizer.padding_side = "left"
def batch_translate_and_score(source_texts, target_references, direction, batch_size=BATCH_SIZE):
    sys_prompt = sys_prompt_en_vi if direction == "en_vi" else sys_prompt_vi_en
    hypotheses = []
    model.eval()
    
    for i in range(0, len(source_texts), batch_size):
        batch = source_texts[i : i + batch_size]
        
        batch_prompts = []
        for text in batch:
            messages = [{"role": "system", "content": sys_prompt}, {"role": "user", "content": text}]
            prompt_str = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            batch_prompts.append(prompt_str)
            
        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_seq_length 
        ).to("cuda")

        src_lens = inputs.attention_mask.sum(dim=1)
        max_src_len = int(src_lens.max())
        DYNAMIC_MAX_NEW_TOKENS = min(int(max_src_len * 1.5), MAX_SAFE_NEW_TOKENS)

        with torch.no_grad():           
            outputs = model.generate(
                input_ids = inputs.input_ids,
                attention_mask = inputs.attention_mask,
                max_new_tokens = DYNAMIC_MAX_NEW_TOKENS,
                do_sample = False,
                num_beams = 1,
                use_cache = True,
                pad_token_id = tokenizer.pad_token_id 
            )
        
        input_lengths = inputs.attention_mask.sum(dim=1)
        
        for idx, output_seq in enumerate(outputs):
            generated_tokens = output_seq[input_lengths[idx]:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
            hypotheses.append(decoded)
            
        print(
            f"   -> Đã dịch {direction}: {i + len(batch)}/{len(source_texts)} câu "
            f"(Max new tokens: {DYNAMIC_MAX_NEW_TOKENS})",
            end='\r'
        )
        
    bleu = sacrebleu.corpus_bleu(hypotheses, target_references)
    return bleu.score, hypotheses


print("\n" + "="*60)
print(f"BẮT ĐẦU TÍNH BLEU SCORE CUỐI CÙNG (Tối ưu hóa tốc độ)")
print(f"   -> Batch Size: {BATCH_SIZE}, Max Tokens: {MAX_SAFE_NEW_TOKENS}")
print("="*60)

# A. Chiều 1: Anh -> Việt (En -> Vi)
bleu_en_vi, hypotheses_en_vi = batch_translate_and_score(valid_en, vi_references, "en_vi", batch_size=BATCH_SIZE)
print(f"\n\n KẾT QUẢ BLEU SCORE (ANH -> VIỆT): {bleu_en_vi:.2f}")

# # B. Chiều 2: Việt -> Anh (Vi -> En)
# bleu_vi_en, hypotheses_vi_en = batch_translate_and_score(valid_vi, en_references, "vi_en", batch_size=BATCH_SIZE)
# print(f"\n\n KẾT QUẢ BLEU SCORE (VIỆT -> ANH): {bleu_vi_en:.2f}")

# print("\n" + "="*60)
# print(f" BLEU SCORE TRUNG BÌNH CUỐI CÙNG: {(bleu_en_vi + bleu_vi_en) / 2:.2f}")
# print("="*60)